In [1]:
import json
import yaml
import requests
import folium

from datetime import datetime
from pymongo import MongoClient
import logging
logging.basicConfig(filename='log.log', level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)


import random

In [2]:
def coneccionDB(mongo_uri:str,database_name:str,collection_name:str):
    "coneccion a una colecccion en base de datos"
    try:
        client = MongoClient(mongo_uri)
        db = client[database_name]
        collection = db[collection_name]
        print(f'coneccion exitosa a la coleccion: {database_name}.{collection_name}')
        return collection 
    except Exception as e:
        logging.warning(e)

## Weather

In [3]:
def GetWeather(coord:list):
    """Coordinates coord= [lat, lon], where lat, lon is latitude and longitude in type float. \n
    Example: [19.29586, -97.768889]. \n
    Invalid formats: 18°55'22.8\" N 98°23'44.3\"W"""
    lat= coord[0]
    lon = coord[1]
    API_key=config['API_key']
    url = f'https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API_key}&units=metric'
    response = requests.get(url)
    data_weather = response.json()
    return data_weather   

In [4]:
def verification(diccionario, valor):

    for calidad, rango in diccionario.items():
        if 'min' in rango and 'max' in rango:
            if rango['min'] <= valor < rango['max']:
                return calidad
        elif 'min' in rango and valor >= rango['min']:
            return calidad

    return None

### Wind

|Qualitative name   |Index  |     wind      |
|-------------------|-------|---------------|
|Calmed down 	    |1 	    |[0;1.4) 	    |
|Soft breeze        |2 	    |[1.4; 5.4) 	|
|Moderate Breeze    |3 	    |[5.4; 10.6) 	|
|Strong wind        |4 	    |[10.6; 17)     |
|Storm   	        |5 	    |⩾17 	        |


Unidad: $m/s$   

In [5]:
data_weather = GetWeather(config['cities']['Atlixco'])
wind_speed = data_weather['wind']['speed']
wind_gust = data_weather['wind']['gust']

windStandars =  config['notifications']['weather']['wind']

In [6]:
def check_wind_speed(diccionario:dict, valor:float):
    Index= {'Calmed down':1, 'Soft breeze':2, 'Moderate Breeze':3, 'Strong wind': 4, 'Storm':5, None: 0}
    ver_txt = verification(diccionario, valor)
    return {'element': 'wind speed','Qualitative name': ver_txt,'Index': Index[ver_txt], 'Value': valor, 'Unit': 'm/s'}

In [7]:
# check_wind_speed(windStandars,wind_speed)

### Temp

|Qualitative name  |Index  |     wind      |
|------------------|-------|---------------|
|Frozen	           |1 	    |[-20,0) 	    |
|Cold              |2 	    |[0,10) 	|
|Moderate          |3 	    |[10,25) 	|
|Warm              |4 	    |[25,35)     |
|Very warm         |5 	    |⩾35        |


Unidad: $°C$   


In [8]:
data_weather = GetWeather(config['cities']['Atlixco'])

temp = data_weather['main']['temp'] #°C
humidity = data_weather['main']['humidity'] # Humidity, %

weatherStandars =  config['notifications']['weather']['temp']

In [9]:
def check_temp(diccionario:dict, valor:float):
    Index= {'Frozen':1, 'Cold':2, 'Moderate':3, 'Warm': 4, 'Very warm':5, None: 0}
    ver_txt = verification(diccionario, valor)
    return {'element': 'temp', 'Qualitative name': ver_txt,'Index': Index[ver_txt], 'Value': valor, 'Unit': '°C'}


In [10]:
# check_temp(weatherStandars,temp)

## Air Pollution




|Qualitative name   |Index  |$SO_2 $    |$NO_2$        |$PM_{10}$       |$PM_{10.5}$   |$O_3$       |$CO$             |
|-------------------|-------|-----------|---------------|---------------|-----------|---------------|-----------------|
|Good 	            |1 	    |[0; 20) 	|[0; 40) 	    |[0; 20) 	    |[0; 10) 	|[0; 60) 	    |[0; 4400)        |
|Fair 	            |2 	    |[20; 80) 	|[40; 70) 	    |[20; 50) 	    |[10; 25) 	|[60; 100) 	    |[4400; 9400)     |
|Moderate 	        |3 	    |[80; 250) 	|[70; 150) 	    |[50; 100) 	    |[25; 50) 	|[100; 140) 	|[9400-12400)     |
|Poor 	            |4 	    |[250; 350) |[150; 200) 	|[100; 200) 	|[50; 75) 	|[140; 180) 	|[12400; 15400)   |
|Very Poor 	        |5 	    |⩾350 	    |⩾200 	       |⩾200 	       | ⩾75 	  |⩾180 	       |⩾15400          |


La unidad de medida es $\mu g /m^3$

fuente: [Open Weather Map](https://openweathermap.org/api/air-pollution)

In [11]:
def GetAirPollution(coord:list):
    """Coordinates coord= [lat, lon], where lat, lon is latitude and longitude in type float. \n
    Example: [19.29586, -97.768889]. \n
    Invalid formats: 18°55'22.8\" N 98°23'44.3\"W"""
    lat= coord[0]
    lon = coord[1]
    API_key=config['API_key']
    url = f'http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid={API_key}&units=metric'
    response = requests.get(url)
    data_weather = response.json()
    return data_weather   

In [12]:
data_pollution = GetAirPollution(config['cities']['Atlixco'])
so2_lvl  = data_pollution['list'][0]['components']['so2']
no2_lvl  = data_pollution['list'][0]['components']['no2']
pm10_lvl = data_pollution['list'][0]['components']['pm10']
pm2_5_lvl = data_pollution['list'][0]['components']['pm2_5']
o3_lvl   = data_pollution['list'][0]['components']['o3']
co_lvl   = data_pollution['list'][0]['components']['co']

AirPollutionStandars =  config['notifications']['AirPollution']

### $SO_2$

In [13]:
def check_so2(diccionario:dict, valor:float):
    Index = {'Good':1, 'Fair':2, 'Moderate':3, 'Poor': 4, 'Very Poor':5, None: 0}
    ver_txt = verification(diccionario, valor)
    return {'element': 'pm10','Qualitative name': ver_txt,'Index': Index[ver_txt], 'Value': valor, 'Unit': 'µg/m^3' }

In [14]:

# check_so2(AirPollutionStandars['so2'],so2_lvl)

### $NO_2$

In [15]:
def check_no2(diccionario:dict, valor:float):
    Index = {'Good':1, 'Fair':2, 'Moderate':3, 'Poor': 4, 'Very Poor':5, None: 0}
    ver_txt = verification(diccionario, valor)
    return {'element': 'no2','Qualitative name': ver_txt,'Index': Index[ver_txt], 'Value': valor, 'Unit': 'µg/m^3' }

In [16]:
# check_no2(AirPollutionStandars['so2'],so2_lvl)

### $PM_{10}$

In [17]:
def check_pm10(diccionario:dict, valor:float):
    Index = {'Good':1, 'Fair':2, 'Moderate':3, 'Poor': 4, 'Very Poor':5, None: 0}
    ver_txt = verification(diccionario, valor)
    return {'element': 'pm10','Qualitative name': ver_txt,'Index': Index[ver_txt], 'Value': valor, 'Unit': 'µg/m^3' }

In [18]:
# check_pm10(AirPollutionStandars['pm10'],pm10_lvl)

## Warnings and notifications

In [19]:
def CreateReport(code_name:str):
    """
    Get a report about weather, air pollution from a specific area. 

    {
        'code name': code_name,
        'time': 'dd-mm-yyyy hh:mm',
        'temp_report'        :      {'element': 'temp', 'Qualitative name': str,'Index': int, 'Value': float, 'Unit': str},
        'wind_speed_report'  :      {'element': 'temp', 'Qualitative name': str,'Index': int, 'Value': float, 'Unit': str},
        'so2_lvl_report'     :      {'element': 'temp', 'Qualitative name': str,'Index': int, 'Value': float, 'Unit': str},
        'no2_lvl_report'     :      {'element': 'temp', 'Qualitative name': str,'Index': int, 'Value': float, 'Unit': str},
        'pm10_lvl_report'    :      {'element': 'temp', 'Qualitative name': str,'Index': int, 'Value': float, 'Unit': str}
    }

    
    """
    coord =config['cities'][code_name]
    #date info
    date_now = datetime.now()
    date_str = date_now.strftime("%d-%m-%Y %H:%M")

    #weather
    data_weather = GetWeather(coord)
    wind_speed   = data_weather['wind']['speed']
    temp         = data_weather['main']['temp']

    weatherStandars =  config['notifications']['weather']['temp']
    #air pollution
    data_pollution = GetAirPollution(coord=coord)
    so2_lvl        = data_pollution['list'][0]['components']['so2']
    no2_lvl        = data_pollution['list'][0]['components']['no2']
    pm10_lvl       = data_pollution['list'][0]['components']['pm10']

    AirPollutionStandars =  config['notifications']['AirPollution']
    #
    temp_report       = check_temp(weatherStandars,temp)
    wind_speed_report = check_wind_speed(windStandars,wind_speed)
    # 
    so2_lvl_report  = check_so2(AirPollutionStandars['so2'],so2_lvl)
    no2_lvl_report  = check_no2(AirPollutionStandars['so2'],no2_lvl)
    pm10_lvl_report = check_pm10(AirPollutionStandars['pm10'],pm10_lvl)

    return {
        'code name': code_name,
        'time':date_str,
        'temp_report':temp_report,
        'wind_speed_report':wind_speed_report,
        'so2_lvl_report':so2_lvl_report,
        'no2_lvl_report': no2_lvl_report,
        'pm10_lvl_report': pm10_lvl_report
    }



In [24]:
def NotificationsAndWarmings(code_name:str, insert=True):
    alerts_collection = coneccionDB(config['connection_url'], config['db_ecoterra'], config['alerts_collection'])

    report = CreateReport(code_name=code_name)

    notifications = {clave: valor for clave, valor in report.items() if isinstance(valor, dict) and valor.get('Index', 0) == config['notifications']['indexNotification']}
    if bool(notifications):
        
        notifications['code name']= report['code name']
        notifications['time']= report['time']
        notifications['type']= 'Notification'
        alerts_collection.insert_one(notifications)

    warming  = {clave: valor for clave, valor in report.items() if isinstance(valor, dict) and valor.get('Index', 0) == config['notifications']['indexAlert']}
    if bool(warming):
        warming['code name']= report['code name']
        warming['time']= report['time']
        warming['type']= 'Warming'
        alerts_collection.insert_one(warming)
    
    if insert:
        report_collection = coneccionDB(config['connection_url'], config['db_ecoterra'], config['weather_collection'])
        report_collection.insert_one(report)
    
    if bool(notifications):
        if bool(warming):
            return notifications, warming
        else: 
            return notifications
    else: 
        if bool(warming):
            return warming
        else:
            return None

In [21]:
# report = CreateReport('Cholula')
# notifications = {clave: valor for clave, valor in report.items() if isinstance(valor, dict) and valor.get('Index', 0) == config['notifications']['indexNotification']}
# CreateReport('Cholula')

In [26]:
NotificationsAndWarmings('Cholula', True)

coneccion exitosa a la coleccion: Ecoterra.alerts
coneccion exitosa a la coleccion: Ecoterra.weather


{'temp_report': {'element': 'temp',
  'Qualitative name': 'Warm',
  'Index': 4,
  'Value': 27.69,
  'Unit': '°C'},
 'code name': 'Cholula',
 'time': '23-02-2024 14:26',
 'type': 'Notification',
 '_id': ObjectId('65d8ff8561e3d3261658ef37')}

## Conection to DataBase

In [27]:
a = ''
print(len(a))

0
